# Image Processing

In [1]:
# imports
import os
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from datetime import datetime
import pytz
from arcgis.geometry import Geometry

def _convert_to_decimal_degress(value):
    """
    Helper function to convert the GPS coordinates stored in the EXIF to degress in float format
    :param value: tuple read from EXIF with DMS coordinate values
    :return: float
    """
    # get the respective degrees, minutes and seconds
    d = float(value[0][0]) / float(value[0][1])
    m = float(value[1][0]) / float(value[1][1])
    s = float(value[2][0]) / float(value[2][1])

    # combine degrees, minutes and seconds into decimal degrees
    return d + (m / 60.0) + (s / 3600.0)


class Img():
    """
    Make it easier to access image properties.
    """
    def __init__(self, image_path):
        self.file = os.path.abspath(image_path)
        self.exif = self._get_exif(image_path)
    
    def _get_exif(self, img):
        """
        Extract the image EXIF data to a dictionary.
        :param img: String path to the image with EXIF data to be parsed.
        :return dict: All tags in very raw format extracted from the image.
        """
        try:
            # get the exif dictionary
            exif_dict = {TAGS.get(tag, tag): value for tag, value in Image.open(img)._getexif().items()}

            # clean up the GPS tags to be human readable as well
            if 'GPSInfo' in exif_dict.keys():
                exif_dict['GPSInfo'] = {GPSTAGS.get(key,key): exif_dict['GPSInfo'][key] for key in exif_dict['GPSInfo'].keys()}

            return exif_dict
        
        except Exception as e:
            
            print(f'ERROR on {img}') 
    
    @property
    def geometry(self):
        """
        Get a point geometry from the GPS dictionary extracted from the image EXIF data.
        :return Point Geometry: Location where the image was captured.
        """
        if self.has_location:
            gps_dict = self.exif['GPSInfo']

            # extract the longitude and latitude values as decimal degrees
            coord_lat = _convert_to_decimal_degress(gps_dict['GPSLatitude'])
            coord_lon = _convert_to_decimal_degress(gps_dict['GPSLongitude'])

            # assign the correct positive or negative value based on hemisphere
            coord_lon = -coord_lon if gps_dict['GPSLongitudeRef'] is 'W' else coord_lon
            coord_lat = -coord_lat if gps_dict['GPSLatitudeRef'] is 'S' else coord_lat

            # create a geometry object from the coordinates
            return Geometry({'x': coord_lon, 'y': coord_lat, 'spatialReference': {'wkid': 4326}})
        
        else:
            return None
        
    @property
    def point(self):
        """
        Get a point geometry from the GPS dictionary extracted from the image EXIF data.
        :return Point Geometry: Location where the image was captured.
        """
        return self.geometry
    
    @property
    def location(self):
        """
        Get a point geometry from the GPS dictionary extracted from the image EXIF data.
        :return Point Geometry: Location where the image was captured.
        """
        return self.geometry
    
    @property
    def gps_datetime(self):
        """
        Get the datetime from the GPS information in the EXIF data.
        :param gps_dict: GPS dictionary extracted from the EXIF dictionary.
        :return datetime: Datetime object when image was captured according to the GPS timestamp.
        """
        if self.has_location:
            gps_dict = self.exif['GPSInfo']

            # extract the hour, minute and second from the GPS information
            gps_time = gps_dict['GPSTimeStamp']

            h = int(gps_time[0][0] / gps_time[0][1])
            m = int(gps_time[1][0] / gps_time[1][1])
            s = int(gps_time[2][0] / gps_time[2][1])

            # extract the year, month and day from the GPS information
            gps_date = [int(val) for val in gps_dict['GPSDateStamp'].split(':')]

            # create a datetime object with the extracted values
            return datetime(gps_date[0], gps_date[1], gps_date[2], h, m, s, tzinfo=pytz.utc)
        
        else:
            return None
        
    @property
    def has_location(self):
        if 'GPSInfo' in self.exif.keys():
            return True
        else:
            return False
        
    @property
    def properites(self):
        return {
            'file': self.file,
            'exif': self.exif,
            'geometry': self.geometry,
            'gps_datetime': self.gps_datetime
        }

In [2]:
# minimal module imports
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd
import imghdr
import os
from arcgis.gis import GIS

# a couple of handy variables and settings to get started
data = r'../data'
data_raw = os.path.join(data, 'raw')
data_raw_image_dir = os.path.join(data_raw, 'images')

In [3]:
# get all the images to be processed
img_file_lst = [
    os.path.abspath(os.path.join(data_raw_image_dir, img)) 
    for img in os.listdir(data_raw_image_dir)
]
img_file_lst = [Img(img) for img in img_file_lst if imghdr.what(img)]

img = img_file_lst[0]
img

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/images'

In [4]:
print(f'unlocated: {len([img.file for img in img_file_lst if not img.has_location])}')
print(f'located: {len([img.file for img in img_file_lst if img.has_location])}')

unlocated: 1
located: 126


In [5]:
df = pd.DataFrame(
    [[img.file, img.gps_datetime, img.geometry] for img in img_file_lst], 
    columns=['file', 'datetime', 'SHAPE']
)
df.spatial.set_geometry('SHAPE')
df.head()

,file,datetime,SHAPE
0,/Users/joel5174/projects/azure_cognitive_weapo...,2019-01-18 20:07:51+00:00,"{""x"": -122.9115752, ""y"": 47.037031899999995, ""..."
1,/Users/joel5174/projects/azure_cognitive_weapo...,2019-01-18 20:10:11+00:00,"{""x"": -122.91179479997223, ""y"": 47.04260389999..."
2,/Users/joel5174/projects/azure_cognitive_weapo...,2019-01-18 20:10:21+00:00,"{""x"": -122.9111895, ""y"": 47.042995699972224, ""..."
3,/Users/joel5174/projects/azure_cognitive_weapo...,2019-01-18 20:08:01+00:00,"{""x"": -122.9122325, ""y"": 47.036903699999996, ""..."
4,/Users/joel5174/projects/azure_cognitive_weapo...,2019-01-18 20:10:41+00:00,"{""x"": -122.90964169997223, ""y"": 47.0434325, ""s..."


In [6]:
mp = GIS().map('Olympia, WA')
mp.basemap = 'dark-gray-vector'
df.spatial.plot(map_widget=mp)
mp.tab_mode = "split-right"
mp

MapView(layout=Layout(height='400px', width='100%'), tab_mode='split-right')